In [1]:
import requests
import os
from dotenv import load_dotenv
import pymongo
import time

In [2]:
def get_current_timestamp_milliseconds():
  """
  Returns the current timestamp in milliseconds since the epoch.
  """
  return int(time.time() * 1000)

In [3]:
load_dotenv() 
LI_URI = os.environ['LI_URI']
LI_VERSION = os.environ['LI_VERSION']
LI_ACCESS_TOKEN = os.environ['LI_ACCESS_TOKEN']

In [4]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
db = db_client['db_li_page_posts']
tb_page_post = db['tb_page_posts']

In [ ]:
def post_2_page(payload):
    #print(url)
    url = LI_URI + 'posts'
    headers = {
        'Authorization': LI_ACCESS_TOKEN,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    try:
        detail = requests.post(url, json=payload, headers=headers)
        return detail   #LI does not return response detail
    except Exception as e:
        print(e)
        return {'error': e}

In [ ]:
#get list of authors from db
def get_1_latest_pots():
    latest_post = tb_page_post.find_one({'shared': 0}, sort=[('lastModifiedAt', -1)])
    return latest_post
#test
the_post = get_1_latest_pots()
if the_post is not None:
    # print(the_post)
    payload = {
        "author": "urn:li:organization:" + os.environ['MY_PAGE_ID'],    #post to my page
        "commentary": '',   #the_post['description'], #duplicated content
        "visibility": "PUBLIC",
        "distribution": {
            "feedDistribution": "MAIN_FEED"
        },
        "lifecycleState": "PUBLISHED",
        # "isReshareDisabledByAuthor": True,
        "reshareContext": {
            "parent": the_post['id']
        }
    }
    result = post_2_page(payload)
    # print(result)
    if 'error' not in result:
        tb_page_post.update_one({'id': the_post['id']}, {'$set': {'shared': 1}})
        print('Finished post to the page with description: ' + the_post['description'][:20] + ' ...')
#end

<Response [201]>
Finished post to the page with description: How have you been wo ...


In [ ]:
#todo auto get RANDOMLY 1 new post from all pages, (maximum 5 repost per day)
def auto_report():
    #1. check how many posts today (<24 hours)

    #2. if today posted < 5 posts:
        #2.1 get 1 new post RANDOMLY, sorted by lastModifiedAt

        #2.2 post it to page

        #2.3 Update to db: shared=1, shared_time=